In [281]:
import numpy as np
import matplotlib.pyplot as plt
import zipfile
try:
  import google.colab
except ImportError:
  from dolfin import *
else:
  try:
    from dolfin import *
  except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    from dolfin import *
%matplotlib inline

In [ ]:
#1. Domain discretisation
!wget "https://github.com/niravshah241/homogenization/archive/refs/heads/main.zip" -O "/tmp/github_files.zip"

zip_ref = zipfile.ZipFile("/tmp/github_files.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

mesh = Mesh("/tmp/homogenization-main/mesh_data/mesh.xml")
subdomains = MeshFunction("size_t", mesh, "/tmp/homogenization-main/mesh_data/mesh_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "/tmp/homogenization-main/mesh_data/mesh_facet_region.xml")

dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

dx_left = dx(1) #Subdomain 1
dx_right = dx(2) #Subdomain 2

ds_left = ds(1) #Left boundary
ds_right = ds(5) #Right boundary
ds_top = ds(2) + ds(4) #Top boundary
ds_botom = ds(3) + ds(6) #Bottom boundary

n = FacetNormal(mesh)

In [283]:
# 2. Create Finite Element space (Lagrange P1)
VT = FunctionSpace(mesh,"CG",1) # Space for temperature field
T_, psi = TrialFunction(VT), TestFunction(VT)
T = Function(VT,name="Temperature") # Temperature field
T_eq = Function(VT,name="Temperature_Equivalent") # Temperature field with equaivalent conductivity

if VT.ufl_element().degree() == 1:
  VQ = VectorFunctionSpace(mesh,"CG",VT.ufl_element().degree()) # Space for heat flux
else:
  VQ = VectorFunctionSpace(mesh,"CG",VT.ufl_element().degree()-1) # Space for heat flux
q_, psi_q = TrialFunction(VQ), TestFunction(VQ)
q = Function(VQ)

In [284]:
# 3. Material and boundary properties, Source term
k_left = 4.5 # Thermal conductivity left subdomain
k_right = 4. # Thermal conductivity right subdomain
T_left = Constant(1800.) # Temperature on left boundary
T_right = Constant(300.) # Temperature on right boundary

In [285]:
# 4. Compute heat flux

def compute_heat_flux(temp_field):
  '''
  Compute the heat flux at given temperature field
  Input:
  temp_field: Function over Functionspace
  Output:
  q: Function over VectorFunctionSpace
  '''
  a_q = inner(q_, psi_q) * dx_left + inner(q_, psi_q) * dx_right
  l_q = inner(-k_left * grad(temp_field), psi_q) * dx_left + inner(-k_right * grad(temp_field), psi_q) * dx_right
  solve(a_q == l_q, q)
  return q

In [ ]:
# 5. Weak formulation
a_T = inner(k_left*grad(T_),grad(psi)) * dx_left + inner(k_right*grad(T_),grad(psi)) * dx_right
l_T = inner(Constant(0.),psi) * dx

bcs = [DirichletBC(VT,T_left,boundaries,1),DirichletBC(VT,T_right,boundaries,5)]

solve(a_T==l_T,T,bcs)

plot(T)
plt.show()

In [ ]:
# 6. Compute equivalent conductivity and solution fields
k_eq = -assemble(compute_heat_flux(T)[0]*dx) / assemble(T.dx(0)*dx)
print(k_eq)

a_T = inner(k_eq*grad(T_),grad(psi)) * dx
l_T = inner(Constant(0.),psi) * dx

bcs = [DirichletBC(VT,T_left,boundaries,1),DirichletBC(VT,T_right,boundaries,5)]

solve(a_T==l_T,T_eq,bcs)

plot(T_eq)
plt.show()

In [ ]:
# 7. Error assessment
error = norm(project(T-T_eq,VT),"L2")/norm(T,"L2")
print(error)